First, load the data,in this notebook we are using the smokoy mountain data as the example. And will use basic AE to get the transformation of the data and do prediction using the same models.

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

2025-01-17 11:50:21.032717: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 11:50:21.036543: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 11:50:21.378401: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 11:50:21.460745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737132621.646439 2793178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737132622.15

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn import svm
import pickle
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available:  0


2025-01-17 11:51:21.755552: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### 1. Load the data and data preprocess
After reading the data, convert the abundance matrix into relative abundance matrix; for the response variables, since we have multiple variables, organize them into an array with each variable as a column.

In [4]:
data,burn_label,un_label,duration_label,ASVs,df=loadData.loadSoilData("../data/count_table/merge_proportion_asv_smoky_moutain.csv")

In [5]:
data = FS.relative_abundance(data)

### 2.AE

In [6]:
from tensorflow.keras import layers
from tensorflow import keras as ks
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Model

# --------------------------------------------------------------------------------------------------#

# To define your model, use the Keras Model Subclassing API.
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='relu'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded



In [7]:
data_train, data_test, y_train, y_test = RunML_continue.split_and_scale_data(data,burn_label)
print(data_train.shape)
print(data_test.shape)

(41, 1768)
(18, 1768)


In [8]:
data_train.shape[1]

1768

In [9]:
latent_dim = 20
shape = data_train.shape[1:]

autoencoder = Autoencoder(latent_dim, shape)
autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())

In [10]:
autoencoder.fit(data_train, data_train,
                epochs=10,
                shuffle=True,
                validation_data=(data_test, data_test))

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 255ms/step - loss: 0.2007 - val_loss: 1.5909
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.2018 - val_loss: 1.5869
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.1990 - val_loss: 1.5836
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.1956 - val_loss: 1.5809
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.1982 - val_loss: 1.5788
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.1963 - val_loss: 1.5770
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.1946 - val_loss: 1.5756
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.1928 - val_loss: 1.5745
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.1948 - val_loss: 1.5735
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.1978 - val_loss: 1.5726


In [11]:
data_AE = autoencoder.encoder(data).numpy()

In [12]:
iter =30
cls = ["RF","SVM", "CatBoost","NB"]

data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [13]:
print(np.shape(data))
print(np.shape(data_AE))

(59, 1768)
(59, 20)


The  function will print out the accuracy and AUC for each dataset using each classifier, and also will return the y_actual, y_predict, y_predprob for future use.

In [14]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

TBB Warning: The number of workers is currently limited to 0. The request for 47 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



             RF_Accuracy   RF_AUC   SVM_Accuracy  SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.780303    0.922917    0.746970    0.84375      0.845455        
AE            0.677273    0.762153    0.557576    0.69375      0.710606        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.887037     0.863636    0.801389  
AE             0.763194     0.575758    0.588079  


In [15]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.779661  1.000000   0.235294    1.000000     0.423886
AllFeatures_SVM       0.745763  0.571429   0.470588    0.857143     0.348896
AllFeatures_CatBoost  0.847458  1.000000   0.470588    1.000000     0.622530
AllFeatures_NB        0.864407  0.846154   0.647059    0.952381     0.655004
AE_RF                 0.677966  0.428571   0.352941    0.809524     0.172957
AE_SVM                0.559322  0.263158   0.294118    0.666667    -0.038010
AE_CatBoost           0.711864  0.500000   0.588235    0.761905     0.334999
AE_NB                 0.576271  0.366667   0.647059    0.547619     0.176362


In [16]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.556061    0.702698    0.556061    0.424444      0.524242        
AE            0.472727    0.466429    0.453030    0.433651      0.456061        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.582540     0.610606    0.605000  
AE             0.459524     0.453030    0.507143  


In [17]:
print(metric.metric_sum(dict_cm_un))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.559322  0.500000   0.230769    0.818182     0.060376
AllFeatures_SVM       0.559322  0.500000   0.346154    0.727273     0.079171
AllFeatures_CatBoost  0.525424  0.400000   0.153846    0.818182    -0.037014
AllFeatures_NB        0.610169  0.551724   0.615385    0.606061     0.219913
AE_RF                 0.474576  0.400000   0.384615    0.545455    -0.070258
AE_SVM                0.457627  0.400000   0.461538    0.454545    -0.083335
AE_CatBoost           0.457627  0.375000   0.346154    0.545455    -0.109547
AE_NB                 0.457627  0.411765   0.538462    0.393939    -0.067916


In [18]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.590909    0.457857    0.507576    0.471607      0.542424        
AE            0.422727    0.319464    0.493939    0.472679      0.443939        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.436429     0.507576    0.516071  
AE             0.315714     0.424242    0.374286  


In [19]:
print(metric.metric_sum(dict_cm_dur))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.593220  0.333333   0.090909    0.891892    -0.027517
AllFeatures_SVM       0.508475  0.230769   0.136364    0.729730    -0.156230
AllFeatures_CatBoost  0.542373  0.222222   0.090909    0.810811    -0.132182
AllFeatures_NB        0.508475  0.360000   0.409091    0.567568    -0.022842
AE_RF                 0.423729  0.227273   0.227273    0.540541    -0.232187
AE_SVM                0.491525  0.318182   0.318182    0.594595    -0.087224
AE_CatBoost           0.440678  0.260870   0.272727    0.540541    -0.185147
AE_NB                 0.423729  0.300000   0.409091    0.432432    -0.153291


In [20]:
shape = data_train.shape[1:]
print(shape)

(1768,)


### fine tune latent dimension and activation function

In [21]:
param_grid = {
            #'input_dim': [X_train_scaled.shape[1]], 
            'latent_dim': [10, 25, 50, 100],
            'activation': ['relu', 'sigmoid', 'tanh']  
        }


In [22]:
best_score = float('inf')
best_model = None
for activation in param_grid['activation']:
    class Autoencoder(Model):
      def __init__(self, latent_dim, shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
          layers.Flatten(),
          layers.Dense(latent_dim, activation=activation),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(tf.math.reduce_prod(shape).numpy(), activation=activation),
          layers.Reshape(shape)
        ])
    
      def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    for latent_dim in param_grid['latent_dim']:
        autoencoder = Autoencoder(latent_dim, shape)
        autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())
        autoencoder.fit(data_train, data_train,
                    epochs=10,
                    shuffle=True,
                    verbose=0,
                    validation_data=(data_test, data_test))
        val_loss = autoencoder.evaluate(data_test, data_test, verbose=0)
        if val_loss < best_score:
            best_score = val_loss
            print(f"activation 1:{activation1},latent_dim:{latent_dim}, val_loss:{val_loss}")
            best_model = autoencoder
        
            

NameError: name 'activation1' is not defined

In [ ]:
data_AE = best_model.encoder(data).numpy()

In [ ]:
data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [ ]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

In [ ]:
print(metric.metric_sum(dict_cm))

In [ ]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

In [ ]:
print(metric.metric_sum(dict_cm_un))

In [ ]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

In [ ]:
print(metric.metric_sum(dict_cm_dur))